In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
from glob import glob

import numpy as np
from PIL import Image

pathname = 'crowd-counting/frames/frames/*.jpg'
images = sorted([image for image in glob(pathname, recursive=False)])
m = min(len(images), 660)
for i in range(m):
    with Image.open(images[i]) as image:
        snapshot = np.array(image)
        if i == 0:
            image_shape = snapshot.shape
            x_upper = np.empty((snapshot.size, m), dtype=np.uint8)
        x_upper[:, i] = snapshot.reshape(-1)

In [ ]:
from timeit import default_timer as timer

from pydmd import DMD

now = timer()
dmd = DMD(svd_rank=-1, exact=False).fit(x_upper)
print(timer() - now)

In [ ]:
from numpy.linalg import norm

index = np.argmin(np.abs(np.log(dmd.eigs)))
dmd.plot_eigs()
print(F'index = {index}')
print(F'amplitude = {dmd.amplitudes[index]}')
print(F'eig = {dmd.eigs[index]}')
print(F'norm(mode) = {norm(dmd.modes[:, index], axis=0)}')

In [ ]:
reconstructed_data = np.outer(dmd.modes[:, index], dmd.dynamics[index])
background = np.round(reconstructed_data.T.real).astype(np.uint8, copy=False)
foreground = x_upper.T - background
foreground[x_upper.T < background] *= -1

In [ ]:
frame = 85
Image.fromarray(x_upper.T[frame].reshape(*image_shape))

In [ ]:
Image.fromarray(background[frame].reshape(*image_shape))

In [ ]:
Image.fromarray(foreground[frame].reshape(*image_shape))